In [ ]:
import numpy as np
import os
import random
from sympy.solvers import solve
from sympy import Symbol
from sklearn.metrics import accuracy_score

In [ ]:
model_1_path = os.path.join(os.getcwd(), "Confidence Score", "WISDM", "wisdm_incep.npy")
model_2_path = os.path.join(os.getcwd(), "Confidence Score", "WISDM", "wisdm_resnet.npy")
model_3_path = os.path.join(os.getcwd(), "Confidence Score", "WISDM", "wisdm_cus.npy")
model_4_path = os.path.join(os.getcwd(), "Confidence Score", "WISDM", "wisdm_den.npy")
model_5_path = os.path.join(os.getcwd(), "Confidence Score", "WISDM", "wisdm_xcep.npy")

test_path = os.path.join(os.getcwd(), "Confidence Score", "WISDM", "y_test_wisdm.npy")

In [ ]:
model_1_score=np.load(model_1_path)
model_2_score=np.load(model_2_path)
model_3_score=np.load(model_3_path)
model_4_score=np.load(model_4_path)
model_5_score=np.load(model_5_path)

y_test=np.load(test_path)

In [ ]:
def Compute_A_3(f,l):
    n = len(f)
    temp_1 = f[0]+f[1]+f[2]
    temp_2 = l * ((f[0] * f[1]) + (f[0] * f[2]) + (f[1] * f[2]))
    temp_3 = (f[0] * f[1] * f[2]) * (l ** (n-1))
    
    return (temp_1 + temp_2 + temp_3)

In [ ]:
def Compute_A_4(f,l):
    n = len(f)
    temp_1 = f[0]+f[1]+f[2] + f[3]
    temp_2 = l * ((f[0] * f[1]) + (f[0] * f[2]) + (f[0] * f[3]) + (f[1] * f[2]) + (f[1] * f[3]) + (f[2] * f[3]))
    temp_3 = (f[0] * f[1] * f[2] * f[3]) * (l ** (n-1))
    
    return (temp_1 + temp_2 + temp_3)

In [ ]:
def Function(x1,x2,x3,x4,x5):
    fuzzy_measure=np.array([x1,x2,x3,x4,x5])
    l=Symbol('l',real=True)
    lam=solve( ( (1+l*fuzzy_measure[0])*(1+l*fuzzy_measure[1])*(1+l*fuzzy_measure[2])*(1+l*fuzzy_measure[3])*(1+l*fuzzy_measure[4]) )-(1+l) , l )
    lam=[i for i in lam if i>0]
    if len(lam)==0:
        return 1
    lam.sort()
    lam=float(lam[0])
    #print("Lam value is : {}".format(lam))
    
    fuzzy_score=np.zeros(shape=model_1_score.shape, dtype=float)
    for i in range(fuzzy_score.shape[0]):
        for j in range(6):
            
            f_score=np.asarray([ model_1_score[i][j], model_2_score[i][j], model_3_score[i][j], model_4_score[i][j], model_5_score[i][j] ])
            
            permute_index=np.argsort(f_score)
            f_score_permuted=f_score[permute_index]
            fuzzy_measure_permuted=fuzzy_measure[permute_index]
            
            A_0=1.0
            A_1 =Compute_A_4(fuzzy_measure_permuted[1:], lam)
            A_2 = Compute_A_3(fuzzy_measure_permuted[2:], lam)
            A_3=( fuzzy_measure_permuted[3] + fuzzy_measure_permuted[4] + lam*fuzzy_measure_permuted[3]*fuzzy_measure_permuted[4] )
            A_4=fuzzy_measure_permuted[4]
            #print("The A_i values are {} , {}, {}".format(A_0,A_1,A_2))
            
            sugeno_score=max([ min(f_score_permuted[0],A_0), min(f_score_permuted[1],A_1), min(f_score_permuted[2],A_2), min(f_score_permuted[3],A_3), min(f_score_permuted[4],A_4) ])
            #print("Integral score is : {}".format(sugeno_score))
            
            fuzzy_score[i][j]=sugeno_score
            
    fuzzy_y_pred=np.argmax(fuzzy_score,axis=1)
    #print("shape is : {}".format(fuzzy_y_pred.shape))
    #print("argmax : {}".format(np.argmax(fuzzy_score[100])))
    acc=accuracy_score(y_test,fuzzy_y_pred)
    
    return (1-acc)

In [ ]:
def Fitness(sample):
    fitness_score=Function(sample[0],sample[1],sample[2], sample[3], sample[4])
    if fitness_score == 0:
        return 9999
    else:
        return abs(1/fitness_score)

In [ ]:
best_sol = None
max_score = 0
for i in range(100):
    
    population=[]
    for p in range(100):
        population.append( (random.uniform(0,1),
                            random.uniform(0,1),
                            random.uniform(0,1),
                            random.uniform(0,1),
                            random.uniform(0,1)) )  
    
    rank=[]
    for p in population:
        fitness=Fitness(p)
        rank.append( (fitness,p))
    rank.sort()
    rank.reverse()
    
    if rank[0][0]>max_score:
        max_score = rank[0][0]
        best_sol = rank[0]
    
    print(rank[0])
    
    best_population=rank[:25]
    
    if best_population[0][0] > 999:
        break
    
    all_candidate=[]
    for item in best_population:
        all_candidate.append( item[1][0])
        all_candidate.append(item[1][1])
        all_candidate.append(item[1][2])
        all_candidate.append(item[1][3])
        all_candidate.append(item[1][4])
    
    new_gen=[]
    for _ in range(100):
        e1=random.choice(all_candidate) * random.uniform(0.0,0.5)
        e2=random.choice(all_candidate) * random.uniform(0.0,0.5)
        e3=random.choice(all_candidate) * random.uniform(0.0,0.5)
        e4=random.choice(all_candidate) * random.uniform(0.0,0.5)
        e5=random.choice(all_candidate) * random.uniform(0.0,0.5)
        new_gen.append( (e1,e2,e3,e4,e5) )
    
    population=new_gen    